In [1]:
#r "nuget: Microsoft.Extensions.AI, 9.9.0"

Installed Packages Microsoft.Extensions.AI, 9.9.0

In [2]:
#r "nuget: Azure.AI.Agents.Persistent, 1.2.0-beta.5"
#r "nuget: Azure.Identity, 1.15.0"
#r "nuget: System.Linq.Async, 6.0.3"
#r "nuget: DotNetEnv, 3.1.1"

Installed Packages Azure.AI.Agents.Persistent, 1.2.0-beta.5 Azure.Identity, 1.15.0 DotNetEnv, 3.1.1 System.Linq.Async, 6.0.3

In [3]:
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Agents.Workflows\bin\Debug\net9.0\Microsoft.Agents.Workflows.dll"

In [4]:
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Agents.AI.AzureAI\bin\Debug\net9.0\Microsoft.Agents.AI.AzureAI.dll"

In [5]:
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Agents.AI\bin\Debug\net9.0\Microsoft.Agents.AI.dll"

In [6]:
using System;
using System.Collections.Generic;
using System.IO;
using System.Linq;
using System.Text;
using System.Text.Json;
using System.Text.Json.Serialization;
using System.Threading.Tasks;
using Azure.AI.Agents.Persistent;
using Azure.Identity;
using Microsoft.Extensions.AI;
using Microsoft.Agents.AI;
using Microsoft.Extensions.Logging;
using Microsoft.Agents.Workflows;
using Microsoft.Agents.Workflows.Reflection;
using DotNetEnv;

In [7]:
// Load environment variables
Env.Load("../../../.env");

var azure_foundry_endpoint = Environment.GetEnvironmentVariable("AZURE_AI_PROJECT_ENDPOINT") ?? throw new InvalidOperationException("FOUNDRY_PROJECT_ENDPOINT is not set.");
var azure_foundry_model_id = "gpt-4o-mini";

var bing_conn_id = Environment.GetEnvironmentVariable("BING_CONNECTION_ID");

In [8]:
bing_conn_id

/subscriptions/ee72069a-4726-4e1a-afe6-7f14e9b9d362/resourceGroups/rg-kinfeylo/providers/Microsoft.CognitiveServices/accounts/kinfeylo-aifoundry-proj-resource/projects/kinfeylo-aifoundry-project/connections/kinfeybingsearch

In [9]:
const string EvangelistInstructions = @"
You are a technology evangelist create a first draft for a technical tutorials.
1. Each knowledge point in the outline must include a link. Follow the link to access the content related to the knowledge point in the outline. Expand on that content.
2. Each knowledge point must be explained in detail.
3. Rewrite the content according to the entry requirements, including the title, outline, and corresponding content. It is not necessary to follow the outline in full order.
4. The content must be more than 200 words.
4. Output draft as Markdown format. set 'draft_content' to the draft content.
5. return result as JSON with fields 'draft_content' (string).";

const string ContentReviewerInstructions = @"
You are a content reviewer and need to check whether the tutorial's draft content meets the following requirements:

1. The draft content less than 200 words, set 'review_result' to 'No' and 'reason' to 'Content is too short'. If the draft content is more than 200 words, set 'review_result' to 'Yes' and 'reason' to 'The content is good'.
2. set 'draft_content' to the original draft content.
3. return result as JSON with fields 'review_result' ('Yes' or  'No' ) and 'reason' (string) and 'draft_content' (string).";

const string PublisherInstructions = @"
You are the content publisher ,run code to save the tutorial's draft content as a Markdown file. Saved file's name is marked with current date and time, such as yearmonthdayhourminsec. Note that if it is 1-9, you need to add 0, such as  20240101123045.md.
";

In [10]:
string OUTLINE_Content =@"
# Introduce AI Agent


## What's AI Agent

https://github.com/microsoft/ai-agents-for-beginners/tree/main/01-intro-to-ai-agents


***Note*** Don's create any sample code 


## Introduce Azure AI Foundry Agent Service 

https://learn.microsoft.com/en-us/azure/ai-foundry/agents/overview


***Note*** Don's create any sample code 


## Microsoft Agent Framework 

https://github.com/microsoft/agent-framework/tree/main/docs/docs-templates


***Note*** Don's create any sample code 
";

In [11]:
var bingGroundingConfig = new BingGroundingSearchConfiguration(bing_conn_id);

BingGroundingToolDefinition bingGroundingTool = new(
    new BingGroundingSearchToolParameters(
        [bingGroundingConfig]
    )
);

In [12]:
var persistentAgentsClient = new PersistentAgentsClient(azure_foundry_endpoint, new AzureCliCredential());

In [13]:
azure_foundry_model_id

gpt-4o-mini

In [14]:
public class ContentResult
{
    [JsonPropertyName("draft_content")]
    public string DraftContent { get; set; } = string.Empty;
}

In [15]:
public  class ReviewResult
{
    [JsonPropertyName("review_result")]
    public string Result { get; set; } = string.Empty;
    [JsonPropertyName("reason")]
    public string Reason { get; set; } = string.Empty;
    [JsonPropertyName("draft_content")]
    public string DraftContent { get; set; } = string.Empty;
}

In [16]:
JsonSerializer.Serialize(ChatResponseFormat.ForJsonSchema(AIJsonUtilities.CreateJsonSchema(typeof(ContentResult)), "ContentResult", "Content Result with DraftContent"))

{"Schema":{"type":"object","properties":{"draft_content":{"type":["string","null"]}}},"SchemaName":"ContentResult","SchemaDescription":"Content Result with DraftContent"}

In [17]:
// Create the three specialized agents
var evangelistMetadata = await persistentAgentsClient.Administration.CreateAgentAsync(
    model: azure_foundry_model_id,
    name: "Evangelist",
    instructions: EvangelistInstructions,
    tools: [bingGroundingTool]
    // responseFormat: new BinaryData(JsonSerializer.Serialize(ChatResponseFormat.ForJsonSchema(AIJsonUtilities.CreateJsonSchema(typeof(ContentResult)), "ContentResult", "Content Result with DraftContent")))
);

var contentReviewerMetadata = await persistentAgentsClient.Administration.CreateAgentAsync(
     model: azure_foundry_model_id,
     name: "ContentReviewer",
     instructions: ContentReviewerInstructions
    //  responseFormat: new BinaryData(JsonSerializer.Serialize(ChatResponseFormat.ForJsonSchema(AIJsonUtilities.CreateJsonSchema(typeof(ReviewResult)), "ReviewResult", "Review Result with review_result, reason and draft_content")))
);

var publisherMetadata = await persistentAgentsClient.Administration.CreateAgentAsync(
    model: azure_foundry_model_id,
    name: "Publisher",
    instructions: PublisherInstructions,
    tools: [new CodeInterpreterToolDefinition()]
);
// var foundryAgent = await persistentAgentsClient.Administration.CreateAgentAsync(model: azure_foundry_model_id);

In [18]:
string evangelist_agentId = evangelistMetadata.Value.Id;
string contentReviewer_agentId = contentReviewerMetadata.Value.Id;
string publisher_agentId = publisherMetadata.Value.Id;

In [20]:
ChatClientAgentOptions EvangelistAgentOptions = new(instructions: EvangelistInstructions)
{
            ChatOptions = new()
            {
                ResponseFormat = ChatResponseFormat.ForJsonSchema(AIJsonUtilities.CreateJsonSchema(typeof(ContentResult)), "ContentResult", "Content Result with DraftContent"),
            }
};

ChatClientAgentOptions ReviewAgentOptions = new(instructions: ContentReviewerInstructions)
{
            ChatOptions = new()
            {
                ResponseFormat = ChatResponseFormat.ForJsonSchema(AIJsonUtilities.CreateJsonSchema(typeof(ReviewResult)), "ReviewResult", "Review Result From DraftContent")
            },
            
};
ChatClientAgentOptions PublisherAgentOptions = new(instructions: PublisherInstructions)
{
            // ChatOptions = new()
            // {
            //     ResponseFormat = ChatResponseFormat.ForJsonSchema(AIJsonUtilities.CreateJsonSchema(typeof(ReviewResult)), "ReviewResult", "Review Result with review_result, reason and draft_content")
            // },
};


In [21]:
AIAgent evangelistagent = await persistentAgentsClient.GetAIAgentAsync(evangelist_agentId,new()
            {
                //ResponseFormat = ChatResponseFormat.ForJsonSchema(AIJsonUtilities.CreateJsonSchema(typeof(ContentResult)), "ContentResult", "Content Result with DraftContent"),
            });
AIAgent contentRevieweragent = await persistentAgentsClient.GetAIAgentAsync(contentReviewer_agentId,new()
            {
                ResponseFormat = ChatResponseFormat.ForJsonSchema(AIJsonUtilities.CreateJsonSchema(typeof(ReviewResult)), "ReviewResult", "Review Result From DraftContent")
            });
AIAgent publisheragent = await persistentAgentsClient.GetAIAgentAsync(publisher_agentId);

In [22]:
// IChatClient evangelistChatClient = persistentAgentsClient.AsIChatClient(evangelistagent.Id)
//             .AsBuilder()
//             .UseFunctionInvocation()
//             .Build();

// IChatClient contentReviewerChatClient = persistentAgentsClient.AsIChatClient(contentRevieweragent.Id)
//             .AsBuilder()
//             .UseFunctionInvocation()
//             .Build();

// IChatClient publisherChatClient = persistentAgentsClient.AsIChatClient(publisheragent.Id)
//             .AsBuilder()
//             .UseFunctionInvocation()
//             .Build();

In [24]:

// ChatClientAgent  evangelistChatAgent = new ChatClientAgent(evangelistChatClient, EvangelistAgentOptions);
// ChatClientAgent  contentReviewerChatAgent = new ChatClientAgent(contentReviewerChatClient, ReviewAgentOptions);
// ChatClientAgent  publisherChatAgent = new ChatClientAgent(publisherChatClient, PublisherAgentOptions);

In [25]:
public class DraftExecutor : ReflectingExecutor<DraftExecutor>, IMessageHandler<ChatMessage, ContentResult>
{
    private readonly AIAgent _evangelistAgent;

    /// <summary>
    /// Creates a new instance of the <see cref="DraftExecutor"/> class.
    /// </summary>
    /// <param name="contentReviewerAgent">The AI agent used for content review</param>
    public DraftExecutor(AIAgent evangelistAgent) : base("DraftExecutor")
    {
        this._evangelistAgent = evangelistAgent;
    }

    public async ValueTask<ContentResult> HandleAsync(ChatMessage message, IWorkflowContext context)
    {
        // Generate a random email ID and store the email content to the shared state

        // Invoke the agent
        
        Console.WriteLine($"DraftExecutor .......loading \n" + message.Text);
        
        var response = await this._evangelistAgent.RunAsync(message);


        Console.WriteLine($"DraftExecutor response: {response.Text}");
        //var contentResult = JsonSerializer.Deserialize<ContentResult>(response.Text);

        ContentResult contentResult = new ContentResult{ DraftContent=Convert.ToString(response) ?? "" };

        Console.WriteLine($"DraftExecutor generated content: {contentResult.DraftContent}");

        // ContentResult contentResult = new ContentResult{ DraftContent="123" };

        return contentResult;
    }
}

In [26]:
public class ContentReviewExecutor : ReflectingExecutor<ContentReviewExecutor>, IMessageHandler<ContentResult, ReviewResult>
{
    private readonly AIAgent _contentReviewerAgent;

    /// <summary>
    /// Creates a new instance of the <see cref="ContentReviewExecutor"/> class.
    /// </summary>
    /// <param name="contentReviewerAgent">The AI agent used for content review</param>
    public ContentReviewExecutor(AIAgent contentReviewerAgent) : base("ContentReviewExecutor")
    {
        this._contentReviewerAgent = contentReviewerAgent;
    }

    public async ValueTask<ReviewResult> HandleAsync(ContentResult content, IWorkflowContext context)
    {
        // Generate a random email ID and store the email content to the shared state

        // Invoke the agent

        Console.WriteLine($"ContentReviewExecutor .......loading");
        var response = await this._contentReviewerAgent.RunAsync(content.DraftContent);
        var reviewResult = JsonSerializer.Deserialize<ReviewResult>(response.Text);
        Console.WriteLine($"ContentReviewExecutor review result: {reviewResult.Result}, reason: {reviewResult.Reason}");

        return reviewResult;
    }
}

In [27]:
public class HandleReviewExecutor() : ReflectingExecutor<HandleReviewExecutor>("HandleReviewExecutor"), IMessageHandler<ReviewResult>
{
    /// <summary>
    /// Simulate the handling of review message.
    /// </summary>
    public async ValueTask HandleAsync(ReviewResult review, IWorkflowContext context)
    {
        if (review.Result == "Yes")
        {
            await context.YieldOutputAsync($"Yes");
        }
        else
        {
            throw new InvalidOperationException("The draft content is not good, cannot publish.");
        }
    }
}

In [28]:
public class PublishExecutor : ReflectingExecutor<PublishExecutor>, IMessageHandler<ReviewResult>
{
    private readonly AIAgent _publishAgent;

    /// <summary>
    /// Creates a new instance of the <see cref="PublishExecutor"/> class.
    /// </summary>
    /// <param name="publishAgent">The AI agent used for publishing</param>
    public PublishExecutor(AIAgent publishAgent) : base("PublishExecutor")
    {
        this._publishAgent = publishAgent;
    }

    /// <summary>
    /// Simulate the sending of an email.
    /// </summary>
    public async ValueTask HandleAsync(ReviewResult review, IWorkflowContext context)
    {
        Console.WriteLine($"PublishExecutor .......loading");
        var response = await this._publishAgent.RunAsync(review.DraftContent);
        Console.WriteLine($"Response from PublishExecutor: {response.Text}");
        await context.YieldOutputAsync($"Publishing result: {response.Text}");
    }
}

In [29]:
public class SendReviewExecutor : ReflectingExecutor<SendReviewExecutor>, IMessageHandler<ReviewResult>
{
    public SendReviewExecutor() : base("SendReviewExecutor")
    {
    }

    /// <summary>
    /// Simulate the sending of an email.
    /// </summary>
    public async ValueTask HandleAsync(ReviewResult message, IWorkflowContext context) =>
        await context.YieldOutputAsync($"Draft content sent: {message.Result}");
}

In [30]:
public Func<object?, bool> GetCondition(string expectedResult) =>
        reviewResult => reviewResult is ReviewResult review && review.Result == expectedResult;

In [31]:

var draftExecutor = new DraftExecutor(evangelistagent);
var contentReviewerExecutor = new ContentReviewExecutor(contentRevieweragent);
var publishExecutor = new PublishExecutor(publisheragent);
var sendReviewerExecutor = new SendReviewExecutor();

In [32]:
var reviewExecutor = new HandleReviewExecutor();

In [33]:
var workflow = new WorkflowBuilder(draftExecutor)
            .AddEdge(draftExecutor, contentReviewerExecutor)
            .AddEdge(contentReviewerExecutor, publishExecutor  , condition: GetCondition(expectedResult: "Yes"))
            .AddEdge(contentReviewerExecutor, sendReviewerExecutor  , condition: GetCondition(expectedResult: "No"))
            .Build();



In [34]:
string OUTLINE_Content =@"
# Introduce AI Agent


## What's AI Agent

https://github.com/microsoft/ai-agents-for-beginners/tree/main/01-intro-to-ai-agents


***Note*** Don's create any sample code 


## Introduce Azure AI Foundry Agent Service 

https://learn.microsoft.com/en-us/azure/ai-foundry/agents/overview


***Note*** Don's create any sample code 


## Microsoft Agent Framework 

https://github.com/microsoft/agent-framework/tree/main/docs/docs-templates


***Note*** Don's create any sample code 
";

In [35]:
OUTLINE_Content


# Introduce AI Agent


## What's AI Agent

https://github.com/microsoft/ai-agents-for-beginners/tree/main/01-intro-to-ai-agents


***Note*** Don's create any sample code 


## Introduce Azure AI Foundry Agent Service 

https://learn.microsoft.com/en-us/azure/ai-foundry/agents/overview


***Note*** Don's create any sample code 


## Microsoft Agent Framework 

https://github.com/microsoft/agent-framework/tree/main/docs/docs-templates


***Note*** Don's create any sample code 


In [36]:
string prompt = @"You need to write a  draft based on the following outline and the content provided in the link corresponding to the outline. 
After draft create , the reviewer check it , if it meets the requirements, it will be submitted to the publisher and save it as a Markdown file, 
otherwise need to rewrite draft until it meets the requirements.
The provided outline content and related links is as follows:" + OUTLINE_Content;

In [37]:
Console.WriteLine(prompt);

You need to write a  draft based on the following outline and the content provided in the link corresponding to the outline. 
After draft create , the reviewer check it , if it meets the requirements, it will be submitted to the publisher and save it as a Markdown file, 
otherwise need to rewrite draft until it meets the requirements.
The provided outline content and related links is as follows:
# Introduce AI Agent


## What's AI Agent

https://github.com/microsoft/ai-agents-for-beginners/tree/main/01-intro-to-ai-agents


***Note*** Don's create any sample code 


## Introduce Azure AI Foundry Agent Service 

https://learn.microsoft.com/en-us/azure/ai-foundry/agents/overview


***Note*** Don's create any sample code 


## Microsoft Agent Framework 

https://github.com/microsoft/agent-framework/tree/main/docs/docs-templates


***Note*** Don's create any sample code 



In [38]:
// workflow

In [39]:
var chat = new ChatMessage(ChatRole.User, prompt);

In [40]:
StreamingRun run = await InProcessExecution.StreamAsync(workflow, chat);

In [41]:
await run.TrySendMessageAsync(new TurnToken(emitEvents: true));
string id="";
string messageData="";
await foreach (WorkflowEvent evt in run.WatchStreamAsync().ConfigureAwait(false))
{
    if (evt is AgentRunUpdateEvent executorComplete)
    {
        if(id=="")
        {
            id=executorComplete.ExecutorId;
        }
        if(id==executorComplete.ExecutorId)
        {
            messageData+=executorComplete.Data.ToString();
        }
        else
        {
            id=executorComplete.ExecutorId;
        }
    }
}

Console.WriteLine(messageData);
// await foreach (WorkflowEvent evt in run.WatchStreamAsync().ConfigureAwait(false))
// {
//             if (evt is WorkflowOutputEvent outputEvent)
//             {
//                 Console.WriteLine($"{outputEvent}");
//             }
// }

DraftExecutor .......loading 
You need to write a  draft based on the following outline and the content provided in the link corresponding to the outline. 
After draft create , the reviewer check it , if it meets the requirements, it will be submitted to the publisher and save it as a Markdown file, 
otherwise need to rewrite draft until it meets the requirements.
The provided outline content and related links is as follows:
# Introduce AI Agent


## What's AI Agent

https://github.com/microsoft/ai-agents-for-beginners/tree/main/01-intro-to-ai-agents


***Note*** Don's create any sample code 


## Introduce Azure AI Foundry Agent Service 

https://learn.microsoft.com/en-us/azure/ai-foundry/agents/overview


***Note*** Don's create any sample code 


## Microsoft Agent Framework 

https://github.com/microsoft/agent-framework/tree/main/docs/docs-templates


***Note*** Don's create any sample code 

DraftExecutor response: ```json
{
  "draft_content": "# Understanding AI Agents\n\n## What